In [1]:
import numpy as np
import numba
import sklearn.datasets
import umap
import pynndescent
import hdbscan
import matplotlib.pyplot as plt
import seaborn as sns
import math
import scipy.sparse
import sklearn.metrics

from collections import namedtuple

sns.set(rc={"figure.figsize":(12, 12)})

In [11]:
mnist = sklearn.datasets.fetch_openml("mnist_784")

## Build the Graph


In [111]:
# Construct the models (using V(X) = E(X^2) - E(X)^2 so we can stream through the data)
# Quirk: We use the 2-out neighborhood and accept the double counting of neighbors of neightbors
#        that overlap. The 2-out lets us see further; the double counting biases toward 
#        the coherent local region as opposed to outliers. Also it just works better. Better
#        options or justifications are more than welcome.
@numba.njit(fastmath=True)
def build_models(nn_inds, nn_dists):
    result = np.zeros((nn_inds.shape[0], 5), dtype=np.float32)
    sums = np.zeros(nn_inds.shape[0], dtype=np.float32)
    sums_of_squares = np.zeros(nn_inds.shape[0], dtype=np.float32)
    counts = np.zeros(nn_inds.shape[0], dtype=np.float32)
    # Get sums and counts for the 1-out (not including the points own 1-nn dist)
    for i in range(nn_inds.shape[0]):
        for j in range(1, nn_inds.shape[1]):
            k = nn_inds[i, j]
            if k != i and nn_dists[k] > 0.0: # Skip zero dists since they don't fit the model
                d = nn_dists[k]
                sums[i] += d
                sums_of_squares[i] += d * d
                counts[i] += 1.0
                
    # Total up totals for the 2-out then compute the mean and std
    for i in range(nn_inds.shape[0]):
        count = 0
        for j in range(nn_inds.shape[1]):
            k = nn_inds[i, j]
            result[i, 0] += sums[k]
            result[i, 1] += sums_of_squares[k]
            count += counts[k]
            
        result[i, 0] /= count
        result[i, 1] = np.sqrt(result[i, 1] / count - result[i, 0] ** 2)
    result[:, 2] = sums
    result[:, 3] = sums_of_squares
    result[:, 4] = counts
    
    return result
    
# Create an edge list of the (directed!) prob of being a nearest neighbor
# This amounts to just computing the relevant prob of the relevant normal using erfc from
# the math library.
@numba.njit(fastmath=True)
def build_edges(nn_inds, nn_dists, models, max_total_weight=32.0, min_prob=1e-3):
    result = []
    root_two = np.sqrt(2)
    for i in range(nn_inds.shape[0]):
        mean = models[i, 0]
        std = models[i, 1]
        total_weight = 0.0
        
        for j in range(nn_inds.shape[1]):
            k = nn_inds[i, j]
            if nn_dists[i, j] == 0:
                if i != k:
                    result.append((i, k, 1.0))
                else:
                    continue
                    
            d = nn_dists[i, j]
            if std > 0:
                erfc_input = (d - mean) / (root_two * std)
                val = math.erfc(erfc_input) / 2.0
            else:
                val = 0.0
            

            total_weight += val
            if total_weight > max_total_weight or val < min_prob:
                break
            else:
                result.append((i, k, val))
            
    return result

# Build the graph: compute nearest neighbors (approximately), build models from them, then make an edge
# list and convert that into a scipy sparse matrix.
def construct_prob_graph(data, n_neighbors=30, metric="euclidean", max_total_weight=32.0, min_prob=1e-3, k=1):
    nn_index = pynndescent.NNDescent(data, metric=metric, n_neighbors=2 * n_neighbors)
    nn_inds, nn_dists = nn_index.neighbor_graph
    models = build_models(nn_inds, nn_dists[:, k])
    
    graph_edges = np.asarray(
        build_edges(nn_inds, nn_dists, models, max_total_weight=max_total_weight, min_prob=min_prob)
    )
    result = scipy.sparse.coo_matrix(
        (graph_edges.T[2], (graph_edges.T[0].astype(np.int32), graph_edges.T[1].astype(np.int32))),
        shape=(data.shape[0], data.shape[0])
    )
    result.eliminate_zeros()
    
    return result

## Cluster the graph

In [3]:
RankDisjointSet = namedtuple("DisjointSet", ["parent", "rank"])
SizeDisjointSet = namedtuple("DisjointSet", ["parent", "size"])


@numba.njit()
def ds_rank_create(n_elements):
    return RankDisjointSet(np.arange(n_elements, dtype=np.int32), np.zeros(n_elements, dtype=np.int32))


@numba.njit()
def ds_size_create(n_elements):
    return SizeDisjointSet(np.arange(n_elements, dtype=np.int32), np.ones(n_elements, dtype=np.int32))


@numba.njit()
def ds_find(disjoint_set, x):
    while disjoint_set.parent[x] != x:
        x, disjoint_set.parent[x] = disjoint_set.parent[x], disjoint_set.parent[disjoint_set.parent[x]]

    return x


@numba.njit()
def ds_union_by_rank(disjoint_set, x, y):
    x = ds_find(disjoint_set, x)
    y = ds_find(disjoint_set, y)

    if x == y:
        return

    if disjoint_set.rank[x] < disjoint_set.rank[y]:
        x, y = y, x

    disjoint_set.parent[y] = x
    if disjoint_set.rank[x] == disjoint_set.rank[y]:
        disjoint_set.rank[x] += 1


@numba.njit()
def ds_union_by_size(disjoint_set, x, y):
    x = ds_find(disjoint_set, x)
    y = ds_find(disjoint_set, y)

    if x == y:
        return

    if disjoint_set.size[x] < disjoint_set.size[y]:
        x, y = y, x

    disjoint_set.parent[y] = x
    disjoint_set.size[x] += disjoint_set.size[y]

Next we need single linkage clustering of the graph. That is most easily done by computing a minimum spanning forest, and then processing that into a forest of merge trees. We could use scipy here, but we want flexibility (for forests instead of trees), and they just use Kruskal's algorithm anyway, which is easy to reproduce here. After that we just need the standard process for converting a spanning tree (or forest) to a merge tree (or forest). In practice I think this can all be simplified to one function, since we are doing the merge work in Kruskal's but for now let's keep things as separate standard algorithms.

Note that to use minimum spanning forests, and merge order structures we need things going in the other order than probabilities, so we just use negative logs (after all, it's only order that matters).

In [4]:
@numba.njit()
def kruskal_minimum_spanning_forest(rows, cols, data, n_vertices):
    
    result_row = np.zeros(n_vertices - 1, dtype=np.int64)
    result_col = np.zeros(n_vertices - 1, dtype=np.int64)
    result_data = np.zeros(n_vertices - 1, dtype=np.float32)
    mst_edge_idx = 0
        
    edge_order = np.argsort(data)
    disjoint_set = ds_rank_create(n_vertices)
    
    for n in range(data.shape[0]):
        edge_idx = edge_order[n]
        i = rows[edge_idx]
        j = cols[edge_idx]
        
        i_component = ds_find(disjoint_set, i)
        j_component = ds_find(disjoint_set, j)
        
        if i_component != j_component:
            result_row[mst_edge_idx] = i
            result_col[mst_edge_idx] = j
            result_data[mst_edge_idx] = data[edge_idx]
            mst_edge_idx += 1
            ds_union_by_rank(disjoint_set, i, j)
            
            if mst_edge_idx >= n_vertices - 1:
                break
            
    return result_row[:mst_edge_idx], result_col[:mst_edge_idx], result_data[:mst_edge_idx]

LinkageMergeData = namedtuple("LinkageMergeData", ["parent", "size", "next"])


@numba.njit()
def create_linkage_merge_data(base_size):
    parent = np.full(2 * base_size - 1, -1, dtype=np.intp)
    size = np.concatenate((np.ones(base_size, dtype=np.intp), np.zeros(base_size - 1, dtype=np.intp)))
    next_parent = np.array([base_size], dtype=np.intp)

    return LinkageMergeData(parent, size, next_parent)


@numba.njit()
def linkage_merge_find(linkage_merge, node):
    relabel = node
    while linkage_merge.parent[node] != -1 and linkage_merge.parent[node] != node:
        node = linkage_merge.parent[node]

    linkage_merge.parent[node] = node

    # label up to the root
    while linkage_merge.parent[relabel] != node:
        next_relabel = linkage_merge.parent[relabel]
        linkage_merge.parent[relabel] = node
        relabel = next_relabel

    return node


@numba.njit()
def linkage_merge_join(linkage_merge, left, right):
    linkage_merge.size[linkage_merge.next[0]] = linkage_merge.size[left] + linkage_merge.size[right]
    linkage_merge.parent[left] = linkage_merge.next[0]
    linkage_merge.parent[right] = linkage_merge.next[0]
    linkage_merge.next[0] += 1


@numba.njit()
def msf_to_linkage_forest(sorted_mst, n_samples=-1):
    result = np.empty((sorted_mst.shape[0], sorted_mst.shape[1] + 1))

    if n_samples < 0:
        n_samples = sorted_mst.shape[0] + 1
        
    linkage_merge = create_linkage_merge_data(n_samples)

    for index in range(sorted_mst.shape[0]):

        left = np.intp(sorted_mst[index, 0])
        right = np.intp(sorted_mst[index, 1])
        delta = sorted_mst[index, 2]

        left_component = linkage_merge_find(linkage_merge, left)
        right_component = linkage_merge_find(linkage_merge, right)

        if left_component > right_component:
            result[index][0] = left_component
            result[index][1] = right_component
        else:
            result[index][1] = left_component
            result[index][0] = right_component

        result[index][2] = delta
        result[index][3] = linkage_merge.size[left_component] + linkage_merge.size[right_component]

        linkage_merge_join(linkage_merge, left_component, right_component)

    return result

def merge_forest_from_graph(prob_graph):
    
    if scipy.sparse.isspmatrix_coo(prob_graph):
        graph_for_linkage = prob_graph
    else:
        graph_for_linkage = prob_graph.tocoo()
        
    msf = kruskal_minimum_spanning_forest(
        graph_for_linkage.row, graph_for_linkage.col, -np.log(graph_for_linkage.data), graph_for_linkage.shape[0]
    )
    msf = np.vstack(msf).T
    merge_forest = msf_to_linkage_forest(msf, graph_for_linkage.shape[0])
    
    return merge_forest

Next we need to be able to condense trees, and also forests. We can just use the standard ``condense_tree`` code (see fast_hdbscan) with some minor modifications where the number of points isn't automatically/implicitly encoded in the merge tree structure. We then supplement that with the ability to extract out roots of the merge forest, and then simply run ``condense_tree`` for each tree in our merge forest, and glue the whole thing together when done.

In [5]:
@numba.njit()
def bfs_from_hierarchy(hierarchy, bfs_root, num_points):
    to_process = [bfs_root]
    result = []

    while to_process:
        result.extend(to_process)
        next_to_process = []
        for n in to_process:
            if n >= num_points:
                i = n - num_points
                next_to_process.append(int(hierarchy[i, 0]))
                next_to_process.append(int(hierarchy[i, 1]))
        to_process = next_to_process

    return result

@numba.njit()
def eliminate_branch(branch_node, parent_node, lambda_value, parents, children, lambdas, sizes, idx, ignore, hierarchy,
                     num_points):
    if branch_node < num_points:
        parents[idx] = parent_node
        children[idx] = branch_node
        lambdas[idx] = lambda_value
        idx += 1
    else:
        for sub_node in bfs_from_hierarchy(hierarchy, branch_node, num_points):
            if sub_node < num_points:
                children[idx] = sub_node
                parents[idx] = parent_node
                lambdas[idx] = lambda_value
                idx += 1
            else:
                ignore[sub_node] = True

    return idx


CondensedTree = namedtuple('CondensedTree', ['parent', 'child', 'lambda_val', 'child_size'])

@numba.njit()
def neg_exp(x):
    return np.exp(-x)

@numba.njit(fastmath=True)
def condense_tree(hierarchy, min_cluster_size=10, root=-1, num_points=-1, next_label=-1, lambda_func=neg_exp):
    
    if root < 0:
        root = 2 * hierarchy.shape[0]
        
    if num_points < 0:
        num_points = hierarchy.shape[0] + 1
        
    if next_label < 0:
        next_label = num_points

    node_list = bfs_from_hierarchy(hierarchy, root, num_points)

    relabel = np.zeros(root + 1, dtype=np.int64)
    relabel[root] = next_label
    
    next_label += 1

    parents = np.ones(root, dtype=np.int64)
    children = np.empty(root, dtype=np.int64)
    lambdas = np.empty(root, dtype=np.float32)
    sizes = np.ones(root, dtype=np.int64)

    ignore = np.zeros(root + 1, dtype=np.bool8)

    idx = 0

    for node in node_list:
        if ignore[node] or node < num_points:
            continue

        parent_node = relabel[node]
        l, r, d, _ = hierarchy[node - num_points]
        left = np.int64(l)
        right = np.int64(r)
        lambda_value = lambda_func(d)

        left_count = np.int64(hierarchy[left - num_points, 3]) if left >= num_points else 1
        right_count = np.int64(hierarchy[right - num_points, 3]) if right >= num_points else 1

        # The logic here is in a strange order, but it has non-trivial performance gains ...
        # The most common case by far is a singleton on the left; and cluster on the right take care of this separately
        if left < num_points and right_count >= min_cluster_size:
            relabel[right] = parent_node
            parents[idx] = parent_node
            children[idx] = left
            lambdas[idx] = lambda_value
            idx += 1
        # Next most common is a small left cluster and a large right cluster: 
        #   relabel the right node; eliminate the left branch
        elif left_count < min_cluster_size and right_count >= min_cluster_size:
            relabel[right] = parent_node
            idx = eliminate_branch(left, parent_node, lambda_value, parents, children, lambdas, sizes, idx, ignore,
                                   hierarchy, num_points)
        # Then we have a large left cluster and a small right cluster: relabel the left node; elimiate the right branch
        elif left_count >= min_cluster_size and right_count < min_cluster_size:
            relabel[left] = parent_node
            idx = eliminate_branch(right, parent_node, lambda_value, parents, children, lambdas, sizes, idx, ignore,
                                   hierarchy, num_points)
        # If both clusters are small then eliminate all branches
        elif left_count < min_cluster_size and right_count < min_cluster_size:
            idx = eliminate_branch(left, parent_node, lambda_value, parents, children, lambdas, sizes, idx, ignore,
                                   hierarchy, num_points)
            idx = eliminate_branch(right, parent_node, lambda_value, parents, children, lambdas, sizes, idx, ignore,
                                   hierarchy, num_points)
        # and finally if we actually have a legitimate cluster split, handle that correctly
        else:
            relabel[left] = next_label

            parents[idx] = parent_node
            children[idx] = next_label
            lambdas[idx] = lambda_value
            sizes[idx] = left_count
            next_label += 1
            idx += 1

            relabel[right] = next_label

            parents[idx] = parent_node
            children[idx] = next_label
            lambdas[idx] = lambda_value
            sizes[idx] = right_count
            next_label += 1
            idx += 1

    return CondensedTree(parents[:idx], children[:idx], lambdas[:idx], sizes[:idx])

@numba.njit()
def merge_tree_roots(merge_tree, min_cluster_size, n_samples=-1):
    if n_samples < 0:
        n_samples = merge_tree.shape[0] + 1
        
    is_root = np.ones(merge_tree.shape[0], dtype=np.bool_)
    for i in range(merge_tree.shape[0]):
        if merge_tree[i, 0] >= n_samples:
            is_root[int(merge_tree[i, 0] - n_samples)] = False
        if merge_tree[i, 1] >= n_samples:
            is_root[int(merge_tree[i, 1] - n_samples)] = False
        
    roots = np.nonzero(is_root)[0] + n_samples
    
    result = []
    for i in range(roots.shape[0]):
        if merge_tree[roots[i] - n_samples, 3] > min_cluster_size:
            result.append(roots[i])
        
    return result

#@numba.njit()
def condense_forest(merge_forest, min_cluster_size=10, n_samples=-1, lambda_func=neg_exp):

    if n_samples < 0:
        n_samples = merge_tree.shape[0] + 1
        
    roots = merge_tree_roots(merge_forest, min_cluster_size=min_cluster_size, n_samples=n_samples)
    
    ctrees = []
    next_label = n_samples + 1
    for root in roots:
        ctree = condense_tree(
            merge_forest, 
            min_cluster_size=min_cluster_size, 
            root=root, 
            num_points=n_samples,
            next_label=next_label,
            lambda_func=lambda_func,
        )
        next_label = ctree.parent.max() + 1
        ctrees.append(ctree)
        
    result = CondensedTree(
        np.concatenate([x.parent for x in ctrees]),
        np.concatenate([x.child for x in ctrees]),
        np.concatenate([x.lambda_val for x in ctrees]),
        np.concatenate([x.child_size for x in ctrees]),
    )
    return result

Lastly we need to be able to extract clusters. We could use any technique, but I went with the leaf extraction because it seemed to work better. In principle this means we don't need to condense the whole tree, just the leaves which could save a lot of time/trouble, but other extraction methods are possible so I'll stick with flexibility (and borrowed standard algorithms) for now.

In [6]:
@numba.njit
def extract_leaves(condensed_tree, allow_single_cluster=True):
    n_points = condensed_tree.parent.min()
    n_nodes = max(condensed_tree.parent.max() + 1, n_points + 1)
    leaf_indicator = np.ones(n_nodes, dtype=np.bool_)
    leaf_indicator[:n_points] = False

    for parent, child_size in zip(condensed_tree.parent, condensed_tree.child_size):
        if child_size > 1:
            leaf_indicator[parent] = False

    return np.nonzero(leaf_indicator)[0]

@numba.njit()
def get_cluster_label_vector(
        tree,
        clusters,
):
    root_cluster = tree.parent.min()
    result = np.empty(root_cluster, dtype=np.intp)
    cluster_label_map = {c: n for n, c in enumerate(np.sort(clusters))}

    disjoint_set = ds_rank_create(tree.parent.max() + 1)
    clusters = set(clusters)

    for n in range(tree.parent.shape[0]):
        child = tree.child[n]
        parent = tree.parent[n]
        if child not in clusters:
            ds_union_by_rank(disjoint_set, parent, child)

    for n in range(root_cluster):
        cluster = ds_find(disjoint_set, n)
        if cluster in cluster_label_map:
            result[n] = cluster_label_map[cluster]
        else:
            result[n] = -1

    return result


def labels_from_prob_graph(prob_graph, min_cluster_size):
    
    merge_forest = merge_forest_from_graph(prob_graph)
    condensed_forest = condense_forest(merge_forest, min_cluster_size=min_cluster_size, n_samples=prob_graph.shape[0])
    chosen_clusters = extract_leaves(condensed_forest)
    
    result = get_cluster_label_vector(condensed_forest, chosen_clusters)
    
    return result

## Propagate labels through the graph

The clustering of the graph works well in that it picks out most of the clusters, but it leaves a great deal of data as noise. We can fix that by running a label propagation through the graph. We do still want to keep the ability to label points as noise, and it would be good to keep the propagation soft, so we can do a Bayesian label propagation of probability vectors over possible labels, including a noise label, with a Bayesian update of the distribution at each propagation round. We can then iterate until relative convergence.

In [7]:
# A basic Bayesian label prop with a few quirks
# Quirks: Different vertices have different (weighted) degrees; we normalize so the total outgoing weight is consistent
#         We reassert labels each iteration boosting the prior; this includes a level of noise prior which is a parameter
#         This is not really theoretically sound, but it prevents labels over-running where they shouldn't. We could in
#         theory simply hold labelled vertics fixed; I haven't experimented with that much yet.
@numba.njit(fastmath=True, parallel=True)
def bayesian_label_prop_iteration(
    indptr, indices, probs, vertex_label_probs, labels, degree_correction, prior=10.0, noise_prior_prob=0.0
):
    
    n_rows = indptr.shape[0] - 1
    n_labels = vertex_label_probs.shape[1]
    vertex_votes = np.zeros_like(vertex_label_probs, dtype=np.float64)
    norms = np.zeros(n_rows)
    for i in numba.prange(n_rows):
        if labels[i] >= 0:
            vertex_votes[i] = prior * vertex_label_probs[i]
            norms[i] += prior
        else:
            vertex_votes[i] = noise_prior_prob * vertex_label_probs[i]
            norms[i] += noise_prior_prob
#         if labels[i] >= 0:
#             vertex_votes[i, labels[i]] = 1.0
#             norms[i] += 1.0
#         else:
#             vertex_votes[i, -1] += noise_prior_prob
#             norms[i] += noise_prior_prob
        
    for i in numba.prange(n_rows):
        for k in range(indptr[i], indptr[i+1]):
            j = indices[k]
            prob = probs[k] * degree_correction[i]
            
            for l in range(n_labels):
                vertex_votes[i, l] += prob * vertex_label_probs[j, l]
                
            norms[i] += prob
        
    for i in numba.prange(n_rows):
        if norms[i] > 0.0:
            vertex_votes[i] /= norms[i]
        else:
            vertex_votes[i, -1] = 1.0
            
    return vertex_votes


def bayesian_label_prop(prob_graph, label_vector, noise_prior_prob=0.0, max_iter=100, tolerance=1e-5, prior=10.0):
    if not scipy.sparse.isspmatrix_csr(prob_graph):
        prob_graph = prob_graph.tocsr()
        
    vertex_label_probs = np.zeros((prob_graph.shape[0], label_vector.max() + 2), dtype=np.float64)
    for i in range(vertex_label_probs.shape[0]):
        vertex_label_probs[i, label_vector[i]] = 1.0
        
    degree_correction = 1.0 / (np.squeeze(np.array(prob_graph.sum(axis=0))) + 1e-3)
    for i in range(max_iter):
        next_vertex_label_probs = bayesian_label_prop_iteration(
            prob_graph.indptr, 
            prob_graph.indices, 
            prob_graph.data, 
            vertex_label_probs, 
            label_vector,
            degree_correction,
            noise_prior_prob=noise_prior_prob,
            prior=prior,
        )
        
        total_change = np.sqrt(np.sum((vertex_label_probs - next_vertex_label_probs)**2))
        
        if total_change / prob_graph.shape[0] < tolerance:
            return next_vertex_label_probs
        else:
            vertex_label_probs = next_vertex_label_probs
            
    return vertex_label_probs

## Putting it all together

We can now put together a clustering algorithm from these pieces. This follows exactly the sections: create the graph; cluster the graph; propagate the initial cluster labels through the graph.

In [8]:
def clusters(
    data,
    metric="euclidean",
    n_neighbors=30,
    min_cluster_size=10,
    max_total_weight=64.0, 
    min_prob=1e-3,
    prior=10.0,
    noise_prior_prob=0.0, 
    max_iter=100, 
    tolerance=1e-5,
    k=1,
):
    prob_graph = construct_prob_graph(
        data, 
        n_neighbors=n_neighbors, 
        metric=metric, 
        max_total_weight=max_total_weight, 
        min_prob=min_prob,
        k=k,
    )
    label_vector = labels_from_prob_graph(prob_graph, min_cluster_size)
    cluster_prob_vector = bayesian_label_prop(
        prob_graph, label_vector, 
        noise_prior_prob=noise_prior_prob, 
        max_iter=max_iter, 
        tolerance=tolerance,
        prior=prior,
    )
    result = np.argmax(cluster_prob_vector, axis=1)
    result[result == label_vector.max() + 1] = -1
    
    return result

In [112]:
data = mnist.data
metric="euclidean"
n_neighbors=30
min_cluster_size=800
max_total_weight=64.0 
min_prob=1e-3
prior=10.0
noise_prior_prob=0.01 
max_iter=100 
tolerance=1e-5
k=1

In [113]:
prob_graph = construct_prob_graph(
    data, 
    n_neighbors=n_neighbors, 
    metric=metric, 
    max_total_weight=max_total_weight, 
    min_prob=min_prob,
    k=k,
)
label_vector = labels_from_prob_graph(prob_graph, min_cluster_size)
cluster_prob_vector = bayesian_label_prop(
    prob_graph, label_vector, 
    noise_prior_prob=noise_prior_prob, 
    max_iter=max_iter, 
    tolerance=tolerance,
    prior=prior,
)
result = np.argmax(cluster_prob_vector, axis=1)
result[result == label_vector.max() + 1] = -1

In [114]:
sklearn.metrics.adjusted_rand_score(mnist.target[result >= 0], result[result >= 0])

0.9272125561175752

In [115]:
np.sum(result >= 0) / result.shape[0]

0.9525428571428571

In [116]:
which_to_keep = np.array([x<=0.9 for x in np.random.random_sample(data.shape[0])])
data0 = data[which_to_keep]
data_new = data[~which_to_keep]

In [117]:
%%time
nn_index = pynndescent.NNDescent(data0, metric=metric, n_neighbors=2 * n_neighbors, verbose=True)
nn_inds, nn_dists = nn_index.neighbor_graph

Tue Aug 29 12:20:36 2023 Building RP forest with 21 trees
Tue Aug 29 12:20:37 2023 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
CPU times: user 2min 43s, sys: 24.8 s, total: 3min 8s
Wall time: 12.8 s


In [135]:
nn_dists.shape

(62859, 60)

In [119]:
models = build_models(nn_inds, nn_dists[:, k])

In [120]:
models

array([[1.09605090e+03, 1.22266335e+02, 6.71438125e+04, 7.73606560e+07,
        5.90000000e+01],
       [9.33732483e+02, 1.03214088e+02, 5.39985625e+04, 4.99931440e+07,
        5.90000000e+01],
       [4.17396088e+02, 6.56829453e+01, 2.49469668e+04, 1.07984160e+07,
        5.90000000e+01],
       ...,
       [9.13740051e+02, 1.28390305e+02, 5.59037969e+04, 5.40132400e+07,
        5.90000000e+01],
       [9.94172180e+02, 1.43832703e+02, 6.25895977e+04, 6.74484000e+07,
        5.90000000e+01],
       [9.74423828e+02, 1.37074188e+02, 5.96473047e+04, 6.17504640e+07,
        5.90000000e+01]], dtype=float32)

In [121]:
graph_edges = np.asarray(
        build_edges(nn_inds, nn_dists, models, max_total_weight=max_total_weight, min_prob=min_prob)
    )

In [122]:
result = scipy.sparse.coo_matrix(
    (graph_edges.T[2], (graph_edges.T[0].astype(np.int32), graph_edges.T[1].astype(np.int32))),
    shape=(data.shape[0], data.shape[0])
)
result.eliminate_zeros()

In [127]:
neighbors[0]

array([[48688, 41441,  3958, ...,  4278, 57903, 24554],
       [17996, 21492, 36138, ..., 57333, 18197, 37415],
       [58114, 14695, 10965, ..., 11169, 44819, 56331],
       ...,
       [54184,  5621,  1325, ...,  6185, 49496, 53634],
       [ 1946, 34142, 33163, ..., 16267,  9708, 17132],
       [51920, 38317, 25115, ..., 19537, 28004, 61676]], dtype=int32)

In [124]:
neighbors = nn_index.query(data_new, k = (2 * n_neighbors) - 1)

Tue Aug 29 12:22:18 2023 Worst tree score: 0.88798740
Tue Aug 29 12:22:18 2023 Mean tree score: 0.89522810
Tue Aug 29 12:22:18 2023 Best tree score: 0.90181199
Tue Aug 29 12:22:19 2023 Forward diversification reduced edges from 3771540 to 624879
Tue Aug 29 12:22:19 2023 Reverse diversification reduced edges from 624879 to 624879
Tue Aug 29 12:22:19 2023 Degree pruning reduced edges from 757860 to 757860
Tue Aug 29 12:22:19 2023 Resorting data and graph based on tree order
Tue Aug 29 12:22:19 2023 Compressing index by removing unneeded attributes
Tue Aug 29 12:22:19 2023 Building and compiling search function


In [148]:
new_inds_tmp = np.array([i+models.shape[0] for i in range(data_new.shape[0])])
new_dists_tmp = np.zeros(data_new.shape[0], dtype=np.float32)
new_inds = new_inds_tmp[:, np.newaxis]
new_dists = new_dists_tmp[:, np.newaxis]

In [149]:
nn_inds_new = np.hstack([new_inds, neighbors[0]])
nn_dists_new = np.hstack([new_dists, neighbors[1]])

In [166]:
# Construct the models (using V(X) = E(X^2) - E(X)^2 so we can stream through the data)
# Quirk: We use the 2-out neighborhood and accept the double counting of neighbors of neightbors
#        that overlap. The 2-out lets us see further; the double counting biases toward 
#        the coherent local region as opposed to outliers. Also it just works better. Better
#        options or justifications are more than welcome.
@numba.njit(fastmath=True)
def build_new_models(nn_inds_new, nn_dists, models):
    new_result = np.zeros((nn_inds_new.shape[0], 5), dtype=np.float32)
    sums = np.zeros(nn_inds_new.shape[0], dtype=np.float32)
    sums_of_squares = np.zeros(nn_inds_new.shape[0], dtype=np.float32)
    counts = np.zeros(nn_inds_new.shape[0], dtype=np.float32)
    # Get sums and counts for the 1-out (not including the points own 1-nn dist)
    for i in range(nn_inds_new.shape[0]):
        for j in range(1, nn_inds_new.shape[1]):
            k = nn_inds_new[i, j]
            if k != i and nn_dists[k] > 0.0: # Skip zero dists since they don't fit the model
                d = nn_dists[k]
                sums[i] += d
                sums_of_squares[i] += d * d
                counts[i] += 1.0
 
    new_result[:, 2] = sums
    new_result[:, 3] = sums_of_squares
    new_result[:, 4] = counts
    
    sums = np.append(models[:,2], sums) 
    sums_of_squares = np.append(models[:,3], sums_of_squares) 
    counts = np.append(models[:,4], counts) 
    
    # Total up totals for the 2-out then compute the mean and std
    for i in range(nn_inds_new.shape[0]):
        count = 0
        for j in range(nn_inds_new.shape[1]):
            k = nn_inds_new[i, j]
            new_result[i, 0] += sums[k]
            new_result[i, 1] += sums_of_squares[k]
            count += counts[k]
            
        new_result[i, 0] /= count
        new_result[i, 1] = np.sqrt(new_result[i, 1] / count - new_result[i, 0] ** 2)
    
    # result = np.vstack([models, new_result])
        
    return new_result

In [167]:
models_new = build_new_models(nn_inds_new, nn_dists[:, k], models)

In [181]:
graph_edges_new = np.asarray(
        build_edges(nn_inds_new, nn_dists_new, models_new, max_total_weight=max_total_weight, min_prob=min_prob)
    ) 
graph_edges_new[: ,0] +=  models.shape[0]*np.ones(graph_edges_new.shape[0])

In [184]:
graph_edges_new[graph_edges_new[:,0]==graph_edges_new[:,1], :]

array([[6.2859e+04, 6.2859e+04, 1.0000e+00],
       [6.2859e+04, 6.2859e+04, 1.0000e+00],
       [6.2860e+04, 6.2860e+04, 1.0000e+00],
       ...,
       [6.9998e+04, 6.9998e+04, 1.0000e+00],
       [6.9999e+04, 6.9999e+04, 1.0000e+00],
       [6.9999e+04, 6.9999e+04, 1.0000e+00]])

In [ ]:
def update_prob_graph(new_data, prob_graph, n_neighbors=30, metric="euclidean", max_total_weight=32.0, min_prob=1e-3, k=1):
    nn_index = prob_graph.nn_index.update(new_data)
    pynndescent.NNDescent(data, metric=metric, n_neighbors=2 * n_neighbors)
    nn_inds, nn_dists = nn_index.neighbor_graph
    models = build_models(nn_inds, nn_dists[:, k])
    
    graph_edges = np.asarray(
        build_edges(nn_inds, nn_dists, models, max_total_weight=max_total_weight, min_prob=min_prob)
    )
    result = scipy.sparse.coo_matrix(
        (graph_edges.T[2], (graph_edges.T[0].astype(np.int32), graph_edges.T[1].astype(np.int32))),
        shape=(data.shape[0], data.shape[0])
    )
    result.eliminate_zeros()
    
    return result

In [ ]:
def infer_clusters(
    data,
    prev_
    metric="euclidean",
    n_neighbors=30,
    min_cluster_size=10,
    max_total_weight=64.0, 
    min_prob=1e-3,
    prior=10.0,
    noise_prior_prob=0.0, 
    max_iter=100, 
    tolerance=1e-5,
    k=1,
):
    prob_graph = construct_prob_graph(
        data, 
        n_neighbors=n_neighbors, 
        metric=metric, 
        max_total_weight=max_total_weight, 
        min_prob=min_prob,
        k=k,
    )
    label_vector = labels_from_prob_graph(prob_graph, min_cluster_size)
    cluster_prob_vector = bayesian_label_prop(
        prob_graph, label_vector, 
        noise_prior_prob=noise_prior_prob, 
        max_iter=max_iter, 
        tolerance=tolerance,
        prior=prior,
    )
    result = np.argmax(cluster_prob_vector, axis=1)
    result[result == label_vector.max() + 1] = -1
    
    return result